# Coursera Capstone Project

This notebook will be used for the capstone project.

### Criteria

For **first week**, you will required to submit the following:
* A description of the problem and a discussion of the background. (15 marks)
* A description of the data and how it will be used to solve the problem. (15 marks)

For the **second week**, the final deliverables of the project will be:
* A link to your Notebook on your Github repository, showing your code. (15 marks)
* A full report consisting of all of the following components (15 marks):
    * Introduction where you discuss the business problem and who would be interested in this project.
    * Data where you describe the data that will be used to solve the problem and the source of the data.
    * Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.
    * Results section where you discuss the results.
    * Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.
    * Conclusion section where you conclude the report.
* Your choice of a presentation or blogpost. (10 marks)

In [1]:
# Importing libraries

import pandas as pd
import numpy as np
import random
import requests

# module to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

# modules to work with geodata
import geopandas as gp
from geopandas.tools import geocode
import folium

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import json

# import tools for webscraping
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib

## 1. Problem Description and Background Discussion

### 1.1 Problem Description
As part of the Capstone Project for the Applied Data Science Coursera Course I have chosen to analyze the effectiveness of the Business Improvement Area (BIA) Program of Toronto, ON in Canada. The question I will answer is: **„Are most Venues located near or in Business Improvement Areas or are there clusters of Venues that should be made into BIAs?“** 

### 1.2 Background Discussion
The **Business Improvement Area (BIA)** is an association of commercial property owners and tenants within a defined area who work in partnership with the City to create thriving, competitive, and safe business areas that attract shoppers, diners, tourists, and new businesses. The question is how effective this association and the created Areas are for attracting shoppers, diners, tourists and new business. 

## 2. Data Description 

### 2.1 Description of Data and Data Source
The BIA layer represents the active BIAs in the City of Toronto that has been enacted by Council. Each BIA has been defined by a by-law and is represented by a Board of Management. The layer is updated as BIAs are created, amended or deleted by Council. This file is a polygon file that shows the BIAs Areas. 

The second part of the data for the analysis comes via the Foursquare API. This dataset contains venues located in Toronto, there location, name, venue category and user rating. The information collected will be for all the central neighborhoods in Toronto.

### 2.2 How will the Data be used to solve the Problem
In a first step the location of the venues will be ploted on a map as overlay to the BIAs. This will show if most Venues are located within or very near BIAs and therefor give an answer about the effectivenes in promoting business near or in BIAs. 

In a second step the venues will be clustered depending on there location and the clusters plotted to show whether the clusters are located whitin the BIAs. This will help answer the second part of the question, whether there are clusters of venues in Toronto that are not part of any BIAs. This clusters could selected for the creation of future BIAs.


### Getting the BIAs Data

Via the API provided by the City of Toronto 

In [2]:
# Get the dataset metadata by passing package_id to the package_search endpoint
# For example, to retrieve the metadata for this dataset:

url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/package_show"
params = { "id": "9edb9628-1213-42bd-8352-5c4ed28e9e42"}
response = urllib.request.urlopen(url, data=bytes(json.dumps(params), encoding="utf-8"))
package = json.loads(response.read())

# Get the data by passing the resource_id to the datastore_search endpoint
# See https://docs.ckan.org/en/latest/maintaining/datastore.html for detailed parameters options
# For example, to retrieve the data content for the first resource in the datastore:

for idx, resource in enumerate(package["result"]["resources"]):
    if resource["datastore_active"]:
        url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/datastore_search"
        p = { "id": resource["id"] }
        r = urllib.request.urlopen(url, data=bytes(json.dumps(p), encoding="utf-8"))
        data = json.loads(r.read())
        df_BIAs = pd.DataFrame(data["result"]["records"])
        break
df_BIAs.head()

,_id,AREA_ID,DATE_EFFECTIVE,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry
0,3215,2481875,2020-02-04T17:20:36,26006975,None,115-00,115-00,Rogers Road,Rogers Road,307227.635,4837983.077,-79.469890,43.681791,17568785,351093.855469,5936.862796,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4662..."
1,3216,2481874,2020-02-04T17:20:36,26006974,None,031-02,031-02,Bloor-Yorkville,Bloor-Yorkville,313738.285,4836723.196,-79.389159,43.670401,17568801,918046.484375,6613.691633,"{""type"": ""Polygon"", ""coordinates"": [[[-79.3872..."
2,3217,2481873,2020-02-04T17:20:36,26006973,None,020-01,020-01,Little Italy,Little Italy,311705.037,4835053.901,-79.414394,43.655397,17568817,232341.589844,3917.542802,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4205..."
3,3218,2481872,2020-02-04T17:20:36,26006972,None,042-01,042-01,Liberty Village,Liberty Village,311152.727,4833083.985,-79.421265,43.637670,17568833,797292.066406,4400.913504,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4246..."
4,3219,2481871,2020-02-04T17:20:36,26006971,None,093-01,093-01,Leslieville,Leslieville,318224.026,4835848.463,-79.333555,43.662460,17568849,351302.890625,6457.749078,"{""type"": ""Polygon"", ""coordinates"": [[[-79.3240..."


In [3]:
df_BIAs.shape

(83, 17)

In [4]:
# dropping BIAs that are out of the central area of Toronto
area_names = ['Albion Islington Square', 'Wilson Village', 'Sheppard East Village', 'The Waterfront', 'Emery Village', 'DuKe Heights', 'Kennedy Road', 'Wexford Heights', 'Crossroads of the Danforth']
index_drop = df_BIAs['AREA_NAME'].isin(area_names)
index_drop = index_drop[index_drop == True]
index_drop.index

Int64Index([5, 17, 22, 28, 45, 50, 57, 58, 79], dtype='int64')

In [5]:
df_BIAs = df_BIAs.drop(index_drop.index)
df_BIAs.head()

,_id,AREA_ID,DATE_EFFECTIVE,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry
0,3215,2481875,2020-02-04T17:20:36,26006975,None,115-00,115-00,Rogers Road,Rogers Road,307227.635,4837983.077,-79.469890,43.681791,17568785,351093.855469,5936.862796,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4662..."
1,3216,2481874,2020-02-04T17:20:36,26006974,None,031-02,031-02,Bloor-Yorkville,Bloor-Yorkville,313738.285,4836723.196,-79.389159,43.670401,17568801,918046.484375,6613.691633,"{""type"": ""Polygon"", ""coordinates"": [[[-79.3872..."
2,3217,2481873,2020-02-04T17:20:36,26006973,None,020-01,020-01,Little Italy,Little Italy,311705.037,4835053.901,-79.414394,43.655397,17568817,232341.589844,3917.542802,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4205..."
3,3218,2481872,2020-02-04T17:20:36,26006972,None,042-01,042-01,Liberty Village,Liberty Village,311152.727,4833083.985,-79.421265,43.637670,17568833,797292.066406,4400.913504,"{""type"": ""Polygon"", ""coordinates"": [[[-79.4246..."
4,3219,2481871,2020-02-04T17:20:36,26006971,None,093-01,093-01,Leslieville,Leslieville,318224.026,4835848.463,-79.333555,43.662460,17568849,351302.890625,6457.749078,"{""type"": ""Polygon"", ""coordinates"": [[[-79.3240..."


In [6]:
df_BIAs.shape

(74, 17)

In [7]:
df_BIAs.reset_index(drop = True, inplace = True)

In [11]:
df_neighborhoods = gp.read_file('Data/Neighbourhoods.geojson.json')
df_neighborhoods.head()

,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry
0,5181,25886861,25926662,49885,94,94,Wychwood (94),Wychwood (94),None,None,-79.425515,43.676919,16491505,3.217960e+06,7515.779658,"POLYGON ((-79.43592 43.68015, -79.43492 43.680..."
1,5182,25886820,25926663,49885,100,100,Yonge-Eglinton (100),Yonge-Eglinton (100),None,None,-79.403590,43.704689,16491521,3.160334e+06,7872.021074,"POLYGON ((-79.41096 43.70408, -79.40962 43.704..."
2,5183,25886834,25926664,49885,97,97,Yonge-St.Clair (97),Yonge-St.Clair (97),None,None,-79.397871,43.687859,16491537,2.222464e+06,8130.411276,"POLYGON ((-79.39119 43.68108, -79.39141 43.680..."
3,5184,25886593,25926665,49885,27,27,York University Heights (27),York University Heights (27),None,None,-79.488883,43.765736,16491553,2.541821e+07,25632.335242,"POLYGON ((-79.50529 43.75987, -79.50488 43.759..."
4,5185,25886688,25926666,49885,31,31,Yorkdale-Glen Park (31),Yorkdale-Glen Park (31),None,None,-79.457108,43.714672,16491569,1.156669e+07,13953.408098,"POLYGON ((-79.43969 43.70561, -79.44011 43.705..."


### Plotting the BIA-Areas on a map

In [13]:
# get location for map centering from Downtown Toronto
address = 'Toronto, Downtown'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Downtown are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto Downtown are 43.6541737, -79.38081164513409.


In [17]:
# create a open street map, center it on a location using latitude and longitude and give it a starting zoom factor
m = folium.Map(location = [latitude, longitude], tiles = 'Stamen Toner', zoom_start = 12)

# create a feature group for the map
fg = folium.map.FeatureGroup(name='BIAs').add_to(m)
fg2 = folium.map.FeatureGroup(name='Neighborhoods').add_to(m)

# add geojson data for the BIAs to map
for i in range(len(df_BIAs['geometry'])):
    b = folium.GeoJson(df_BIAs['geometry'][i])
    b.add_child(folium.Popup(df_BIAs['AREA_NAME'][i]))
    fg.add_child(b)
    
# add geojson data for the neighborhoods to map
for i in range(len(df_neighborhoods['geometry'])):
    a = folium.GeoJson(df_neighborhoods['geometry'][i])
    a.add_child(folium.Popup(df_neighborhoods['AREA_NAME'][i]))
    fg2.add_child(a)   
    
    
folium.LayerControl().add_to(m)
    
# display the map
m

### Setting up the API for accessing foursquare data

In [ ]:
CLIENT_ID = '5MEM4YM205NTQBOMWUQX00NHLMW2GJGAV2OPGIHK55JSJKFU' # your Foursquare ID
CLIENT_SECRET = 'XQ34UGNCZTPZWQFKCIVSYLXHK533UR24OSHJ1BKLE2SSZTT3' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

### Getting Data for the Venues around the BIAs
Via a API request the data for the venues in the BIAs Areas are collected and stored in a Data Frame for easy data manipulation. The definied function will get, depending on the location of the BIA, the names of the venues within a that BIA, exact location (latitude, longitude) and venue category. 

In [ ]:
# defining a function to get the Venues Name, Location, Rating and Category via the Foursquare API

def getNearbyVenues(names, latitudes, longitudes, radius=600, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['BIA', 
                  'BIA Latitude', 
                  'BIA Longitude',
                  'Venue ID',
                  'Venue', 
                  'Venue Latitude',
                  'Venue Longitude',
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
bia_venues = getNearbyVenues(names=df_BIAs['AREA_NAME'],
                                   latitudes=df_BIAs['LATITUDE'],
                                   longitudes=df_BIAs['LONGITUDE']
                                  )

In [ ]:
print(bia_venues.shape)
bia_venues.head()

In [ ]:
# Prepare the data for Heatmap
# Ensure you're handing it floats
around_bia_venues['Venue Latitude'] = around_bia_venues['Venue Latitude'].astype(float)
around_bia_venues['Venue Longitude'] = around_bia_venues['Venue Longitude'].astype(float)

# Filter the DF for rows, then columns, then remove NaNs
heat_df = around_bia_venues[['Venue Latitude', 'Venue Longitude']]
heat_df = heat_df.dropna(axis=0, subset=['Venue Latitude','Venue Longitude'])

# List comprehension to make out list of lists
heat_data = [[row['Venue Latitude'],row['Venue Longitude']] for index, row in heat_df.iterrows()]

## Analysing the Data
The collected Data is analised in the following ways:
1. Mapping the Venues around the BIAs
2.
3. 

### 1. Mapping the BIAs and Venues for Overview

In [ ]:
# create map of Toronto with BIAs (blue) and Venues within the BIAs (red)

toronto_map = folium.Map(location = [latitude, longitude], tiles = 'Stamen Toner', zoom_start = 12)

# create a feature group for the map
fg = folium.map.FeatureGroup(name='BIAs').add_to(toronto_map)
fg1 = folium.map.FeatureGroup(name = 'Venues').add_to(toronto_map)
fg4 = folium.map.FeatureGroup(name = 'Density of Venues').add_to(toronto_map)

# add geojson data for the BIAs to map
for i in range(len(df_BIAs['geometry'])):
    b = folium.GeoJson(df_BIAs['geometry'][i])
    b.add_child(folium.Popup(df_BIAs['AREA_NAME'][i]))
    fg.add_child(b)
    
for lat, long, name in zip(df_merged['Venue Latitude'], df_merged['Venue Longitude'], df_merged['Venue']):
    name = folium.Popup(name, parse_html = True)
    c = folium.Circle(
    [lat, long],
    radius = 2,
    popup = name,
    color = 'red',
    fill = True,
    parse_html = False)
    fg1.add_child(c)   
    
    

# Plot heatmap
h = HeatMap(heat_data)
fg4.add_child(h)    
    
folium.LayerControl().add_to(toronto_map)
    
# display the map
toronto_map